# Approximating execution time cycles per Isogeny
In this notebook we compile the target implementation with added code which measures the approximate amount of clock cycles for each isogeny. 

It is just an approximation and the added code also adds some amount of cycles in total, but since we are doing just a small amount of isogenies (15 for dummy implementation or 30 for dummy-free implementation) this approximation suffices. 

For more precise measurements, it is possible to just change the triggers (`#define A2` instead of `A1`), then we could measure the time via processing an oscilloscope trace where rising edges would signify a start of isogeny, which can be then converted to the clock cycles since we know the clock frequency.

In [ ]:
PATH = "/home/xjaros2/Documents/git/csidh-setup/"
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_STM32F3'
BENCH_MODE = 'PROFILE'

SS_VER = 'SS_VER_2_1'
attack_type = "A1"

In [ ]:
%cd $PATH/notebooks
%run ./init.ipynb
%cd $PATH/notebooks

In [ ]:
csidh.action_sleep = 0
csidh.reset_target()
csidh.scope.arm()
csidh.action()

result = ""
while "#" not in result:
    x = csidh.target.read()
    result += x

In [ ]:
lines = result.split("#")[0]
lines = lines.split('\n')
lines = list(filter(None,lines))
lines

In [ ]:
APPROX_TOTAL_EXECUTION_TIME = int(lines[0])

In [ ]:
ISOGENIES = lines[1:]
ISOGENIES[-1]

ISOGENIES = list(map(lambda x: list(map(int, x.split(',')[1:])), ISOGENIES))
ISOGENIES

In [ ]:
APPROX_EXECUTION_TIME = [{
    "degree":degree & 0x7f if degree &(1<<7) else -(degree & 0x7f), # Signs are opposite in the algorithm (compared with SAGE implementation)
    "setup_start": 0,
    "isogeny_start": isogeny_start, 
    "isogeny_end": isogeny_end
} for degree, isogeny_start, isogeny_end in ISOGENIES
]

APPROX_EXECUTION_TIME[0]['setup_start'] = 0
for i in range(1, len(APPROX_EXECUTION_TIME)):
    APPROX_EXECUTION_TIME[i]['setup_start'] = APPROX_EXECUTION_TIME[i-1]['isogeny_end'] + 1
    

In [ ]:
import pandas as pd
APPROX_EXECUTION_TIME = pd.DataFrame(APPROX_EXECUTION_TIME)